In [92]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import pickle
import json

In [93]:
output_file_list = glob("/opt/ml/input/workspace/calc_score/*.csv")
output_file_list.sort()


In [94]:
output_file_list

['/opt/ml/input/workspace/calc_score/1_submission_B4R_9_T10PF.csv',
 '/opt/ml/input/workspace/calc_score/2_submission_DAE_150.csv',
 '/opt/ml/input/workspace/calc_score/3_submission_MVAE_200_rm.csv',
 '/opt/ml/input/workspace/calc_score/4_submission_UB.csv',
 '/opt/ml/input/workspace/calc_score/5_submission_DeepFM.csv',
 '/opt/ml/input/workspace/calc_score/6_submission_FFM.csv',
 '/opt/ml/input/workspace/calc_score/7_submission_50_g0004.csv']

In [95]:
df_list = list()

for file in output_file_list:
    print(file)
    answer = pd.read_csv(file)
    df_list.append(answer)
print(df_list[:1])# [[model1의 답],[model2의 답],,,[model n의 답]]  

# model_ratio={'b4r': 0.55,
#  'dae': 0.2,
#  'mvae': 0.1,
#  'ub': 0.07,
#  'dfm': 0.05,
#  'ffm': 0.03,
#  'rvae': 0.02}

model_ratio={'b4r': 5,
 'dae': 3,
 'mvae': 3,
 'ub': 4,
 'dfm': 0,
 'ffm': 0,
 'rvae': 0,
 'top10' : }


/opt/ml/input/workspace/calc_score/1_submission_B4R_9_T10PF.csv
/opt/ml/input/workspace/calc_score/2_submission_DAE_150.csv
/opt/ml/input/workspace/calc_score/3_submission_MVAE_200_rm.csv
/opt/ml/input/workspace/calc_score/4_submission_UB.csv
/opt/ml/input/workspace/calc_score/5_submission_DeepFM.csv
/opt/ml/input/workspace/calc_score/6_submission_FFM.csv
/opt/ml/input/workspace/calc_score/7_submission_50_g0004.csv
[          user  item
0           11  1196
1           11   780
2           11   480
3           11   344
4           11  4370
...        ...   ...
313595  138493  8961
313596  138493  1907
313597  138493  2628
313598  138493  2571
313599  138493  1022

[313600 rows x 2 columns]]


In [96]:
with open('/opt/ml/input/level2-movie-recommendation-level2-recsys-07/MODELS/DeepFM/item_dict.pickle', 'rb') as fr:
    item_dict = pickle.load(fr)

In [97]:
train_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')
train_df['rating']=1

user_item_matrix = train_df.pivot_table('rating', 'user', 'item').fillna(0)
user_list=user_item_matrix.index.to_list()
item_list=user_item_matrix.columns.to_list()

In [98]:
for df_index in np.arange(len(df_list)):
    df_list[df_index] = df_list[df_index].groupby("user")["item"].apply(list)

In [99]:
df_list[3][11]

[47, 4886, 5418, 8961, 32587, 1682, 1265, 2115, 4963, 1517]

In [100]:
model_ratio = list(model_ratio.values())

In [101]:
model_ratio

[7, 6, 5, 4, 3, 2, 1]

In [102]:
user_recommends = dict()

res_user = list()
res_item = list()

for user in tqdm(user_list):
    vote_list = [0] * 119146
    for df_index in np.arange(len(df_list)):
        ratio = model_ratio[df_index]
        user_recommended10 = df_list[df_index][user]

        for item in user_recommended10:
            vote_list[item] += ratio

    user_recommends = list(np.argpartition(vote_list,-10)[-10:])
    
    users = [user] * 10

    res_user += users
    res_item += user_recommends

result = np.vstack((res_user, res_item)).T
result

100%|██████████| 31360/31360 [04:34<00:00, 114.11it/s]


array([[    11,  37386],
       [    11,   1196],
       [    11,   8644],
       ...,
       [138493,   2628],
       [138493,   1270],
       [138493,   2011]])

In [103]:
assert len(result) == 313600

In [104]:
submission_df = pd.DataFrame(data=result, columns=["user", "item"])
submission_df.to_csv("./submission_weighted_hard_vote3.csv", index=False)

In [105]:
print("testing recall@10...")
# 학습에 사용된 user만 uniq_user에 저장
#uniq_user = list(user_recommends.keys())#
uniq_user = user_list
print (f"Number of users : {len(uniq_user)}")

with open("/opt/ml/input/workspace/final_answer.json", "r") as json_file: #answer.json 경로 지정
    answer = json.load(json_file)

# movielens-20m과 submission을 비교하여 Recall@10 값 계산
#submission_df = pd.read_csv("./submission_weighted_hard_vote.csv")
recall_result = []

# 각 유저마다 recall@10 계산하여 list에 저장
for user in tqdm(uniq_user):
    submission_by_user = submission_df[submission_df['user'] == user]['item']

    #user = int(user)
    hit = 0
    for item in submission_by_user:
        if item in answer[str(user)]:
            hit += 1

    recall_result.append(hit / len(answer[str(user)]))

# 전체 유저의 Recall@10의 평균 출력
print (f"Predicted submission result of Recall@10 = {np.average(recall_result)}")

testing recall@10...
Number of users : 31360


100%|██████████| 31360/31360 [00:28<00:00, 1108.82it/s]

Predicted submission result of Recall@10 = 0.1159305566810214


- 3: Predicted submission result of Recall@10 = 0.1159305566810214